## After GPR-Lidar Fusion

In [11]:
import datetime
import pandas as pd
import numpy as np

## Identify the multiple GPR points within LiDAR cells and find median TWTs
 - be sure to update CSV name
 - remoce random IX and lidar SD column when proper data updated
 - update column names 

In [12]:
#import data change name based on csv file
GPR = pd.read_csv('GPR_ALL.csv')
GPR.head()

#filter by date to seperate 2 data sets
date1 = '2020-02-02'
date2 = '2020-02-04'
GPR['date'] = pd.to_datetime(GPR['date'])
GPR1 = GPR[(GPR['date'] <= date1)] #less than and equal to
GPR1.head()

GPR2 = GPR[(GPR['date'] >= date2)]  #greater than and equal to this date 

print(len(GPR1), ',', len(GPR2))

### THIS IS TEMPORARYYY!! Delete when we have real data
##add fake columns for GPR to test out lidar 
#will delete when have real data

lidarSD = np.ones((len(GPR1),1))
GPR1['lidarSD'] = lidarSD
GPR1.head()

Ix = range(1000)
Ix = np.random.choice(Ix, size=len(GPR1))
GPR1['Ix'] = Ix

#### KEEP HERE
## update columns when we know them with real data ( Ix column is refering to the Lidar cell ID)
GPRLiDAR = GPR1.groupby(['Ix']).median()
del GPRLiDAR['easting']
del GPRLiDAR['northing']

#GPRLiDAR.drop('easting',inplace = True)
GPRLiDAR.head()



1153612 , 395584


/tmp/ipykernel_9488/2638411004.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GPR1['lidarSD'] = lidarSD
/tmp/ipykernel_9488/2638411004.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GPR1['Ix'] = Ix


,Unnamed: 0,twt,lidarSD
Ix,,,
0,573568.0,7.8,1.0
1,580913.0,7.8,1.0
2,569658.0,7.9,1.0
3,570502.0,7.9,1.0
4,566541.0,7.8,1.0


## Calculate snow density 
- eq 1. hs = TWT(v)/2  hs = Lidar snow depth, TWT = gpr two way travel time, V = wave speed
- eq 2. V = C/ sqrt(e),  V = wave speed, C = speed of light (0.3m/ns), e = dielectric permittivity
- eq 3. e = (1+0.845d)^2  Kovacs et al. 1995 d = density (kg/m3 /1000)

re-arrange eq 1.
- V = hs(2)/TWT

re-arrange eq 2.
- e = C^2/V^2

re-arrange eq 3.
- d = (sqrt(e)-1)/0.845

- update lidarSD column name 

In [13]:
#First calculation 1
#V = wave speed
GPRLiDAR['v']= (GPRLiDAR["lidarSD"]*2)/GPR["twt"]

In [14]:
#second Calculation 
#e = dielectric permitivity
GPRLiDAR['e'] = 0.3**2/(GPRLiDAR['v']**2)

In [15]:
#third calculation
# density
# d = (sqrt(e)-1)/0.845
GPRLiDAR['dens'] = (((GPRLiDAR['e'])**(1/2))-1)/0.845
#convert units to kg/m^3
GPRLiDAR['dens'] = GPRLiDAR['dens']*1000

In [16]:
GPRLiDAR.head()

,Unnamed: 0,twt,lidarSD,v,e,dens
Ix,,,,,,
0,573568.0,7.8,1.0,0.240964,1.550025,289.940828
1,580913.0,7.8,1.0,0.240964,1.550025,289.940828
2,569658.0,7.9,1.0,0.240964,1.550025,289.940828
3,570502.0,7.9,1.0,0.240964,1.550025,289.940828
4,566541.0,7.8,1.0,0.240964,1.550025,289.940828


## Simple Median Filter to remove 
- running median filter using a buffer


- this will tell you what variables you have 
%whos

In [20]:
GPRLiDAR['RUNNONGdens'] = GPRLiDAR['dens'].rolling(20).median()

In [21]:
GPRLiDAR.head()

,Unnamed: 0,twt,lidarSD,v,e,dens,RUNNONGdens
Ix,,,,,,,
0,573568.0,7.8,1.0,0.240964,1.550025,289.940828,NaN
1,580913.0,7.8,1.0,0.240964,1.550025,289.940828,NaN
2,569658.0,7.9,1.0,0.240964,1.550025,289.940828,NaN
3,570502.0,7.9,1.0,0.240964,1.550025,289.940828,NaN
4,566541.0,7.8,1.0,0.240964,1.550025,289.940828,NaN
